<a href="https://colab.research.google.com/github/satyam0104/ImageStiching/blob/main/Image_stiching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing libraries
from random import randrange
import matplotlib.pyplot as plt
import numpy as np
import cv2
figsize = (10, 10)


In [ ]:
#Reading images and converting them into gray
img=cv2.imread('1.jpg')
img1=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img_=cv2.imread('2.jpg')
img2=cv2.cvtColor(img_,cv2.COLOR_BGR2GRAY)


## SIFT feature detection and description


In [ ]:
#Creating a object of cv2.SIFT_create() class
feature_extracter=cv2.SIFT_create()
#finding the keypoint and descriptors using chosen extracter(SIFT)
keypoint_1,desc_1=feature_extracter.detectAndCompute(img1,None)
keypoint_2,desc_2=feature_extracter.detectAndCompute(img2,None)

test= cv2.drawKeypoints(img,keypoint_1,None,flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
plt.figure(figsize=figsize)#important if you want to see in original size
plt.imshow(test)
plt.title("keypoints")
plt.show()

## take only unique features


In [ ]:
#The BFMatcher() matches the features which are more similar. 
#When we set parameter k=2, we are asking the knnMatcher to give out 2 best matches for each descriptor.
bf=cv2.BFMatcher()
matches=bf.knnMatch(desc_1,desc_2,k=2)# we pass descriptor here.

#now we will select the best matches and ignore poor matches
#we apply ratio test using the top 2 matches obtained above.
#We consider a match if the ratio defined below is predominantly greater than the specified ratio.
good_and_second_good_match_list = []
for m in matches:
    if m[0].distance/m[1].distance < 0.5:
        good_and_second_good_match_list.append(m)
        
good_match_arr = np.asarray(good_and_second_good_match_list)[:,0]

# show only 30 matches
im_matches = cv2.drawMatchesKnn(img, keypoint_1, img_, keypoint_2,
                                good_and_second_good_match_list[0:30], None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

plt.figure(figsize=(20, 20))
plt.imshow(im_matches)
plt.title("keypoints matches")
plt.show()


## find homography between images


In [ ]:
good_keypoint_1= np.array([keypoint_1[m.queryIdx].pt for m in good_match_arr])
good_keypoint_2= np.array([keypoint_2[m.trainIdx].pt for m in good_match_arr])
H, masked = cv2.findHomography(good_keypoint_1, good_keypoint_2, cv2.RANSAC, 5.0)



In [ ]:
print(H)

## overlay images


In [ ]:
img__warped = cv2.warpPerspective(img_, H, (img.shape[1] + img_.shape[1], img.shape[0]))
img__warped[0:img.shape[0], 0:img.shape[1]] = img

plt.figure(figsize=figsize)
plt.imshow(img__warped)
plt.title("naive warping")
plt.show()


